In [2]:
import os
import tkinter as tk
from tkinter import simpledialog

def main():
    # Initialisierung des GUI-Fensters
    root = tk.Tk()
    root.withdraw()  # Versteckt das Hauptfenster

    # Eingabe des Pfades des Eingabe- und Ausgabeordners
    # Dialog boxen ausbauen
    input_folder_path = simpledialog.askstring("Input", "Eingabe Pfad?")
    output_folder_path = simpledialog.askstring("Input", "Ausgabe Pfad?")
    zI = simpledialog.askstring("Input", "Sollen Zeichen am Anfang ignoriert werden, wenn ja wie viele? Sonst 0. Eingabe muss eine Zahl sein")

    try:
        zeichen_ignorieren = int(zI)
    except ValueError:
        print("Ungültige Eingabe für die Anzahl der zu ignorierenden Zeichen.")
        return

    if not input_folder_path or not output_folder_path:
        print("Ungültiger Pfad.")
        return

    input_folder = os.path.abspath(input_folder_path)
    output_folder = os.path.abspath(output_folder_path)

    if not os.path.isdir(input_folder):
        print("Eingabeordner existiert nicht.")
        return

    if not os.path.isdir(output_folder):
        os.makedirs(output_folder)

    # Map zur Speicherung von Dateinamen und ihren Zeilen
    file_lines_map = {}

    for file_name in os.listdir(input_folder):
        input_file_path = os.path.join(input_folder, file_name)
        if os.path.isfile(input_file_path):
            file_key = file_name[zeichen_ignorieren:] if len(file_name) > zeichen_ignorieren else file_name
            if file_key not in file_lines_map:
                file_lines_map[file_key] = []

            try:
                lines = read_file_with_fallback(input_file_path)
                file_lines_map[file_key].extend(lines)
            except IOError as e:
                print(f"Fehler beim Lesen der Datei {input_file_path}: {e}")

    # Ausgabe der zusammengefügten Dateien
    for file_key, lines in file_lines_map.items():
        output_file_path = os.path.join(output_folder, file_key)
        try:
            with open(output_file_path, 'w', encoding='utf-8') as writer:
                for line in lines:
                    writer.write(line + '\n')
        except IOError as e:
            print(f"Fehler beim Schreiben der Datei {output_file_path}: {e}")

    print("Dateien wurden zusammengefasst.")

def read_file_with_fallback(file_path):
    encodings = ['utf-8', 'latin-1', 'cp1252']  # Liste der zu versuchenden Codierungen
    for encoding in encodings:
        try:
            with open(file_path, 'r', encoding=encoding) as f:
                return [line.strip() for line in f]
        except UnicodeDecodeError:
            continue
    raise UnicodeDecodeError(f"Keine der Codierungen konnte die Datei lesen: {file_path}")

if __name__ == "__main__":
    main()


TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'